In [ ]:
# Imports

import datetime as dt
from functools import cache

import geopandas as gpd
import matplotlib.animation as anim
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.axes import Axes
from tqdm.auto import tqdm

In [ ]:
records = [
    {"country": "Albania", "coverage": 1, "introduced": dt.date(2025, 12, 18)},
    {"country": "Austria", "coverage": 1, "introduced": dt.date(2023, 12, 27)},
    {"country": "Belarus", "coverage": 1, "introduced": dt.date(2026, 2, 3)},
    {"country": "Belgium", "coverage": 1, "introduced": dt.date(2023, 12, 27)},
    {
        "country": "Bosnia and Herzegovina",
        "coverage": 1,
        "introduced": dt.date(2025, 11, 17),
    },
    {"country": "Bulgaria", "coverage": 1, "introduced": dt.date(2025, 10, 24)},
    {"country": "Croatia", "coverage": 1, "introduced": dt.date(2025, 11, 3)},
    {"country": "Czechia", "coverage": 1, "introduced": dt.date(2025, 10, 1)},
    {"country": "Denmark", "coverage": 1, "introduced": dt.date(2021, 7, 25)},
    {"country": "Estonia", "coverage": 1, "introduced": dt.date(2025, 9, 7)},
    {"country": "Faroe Islands", "coverage": 1, "introduced": dt.date(2021, 9, 9)},
    {"country": "Finland", "coverage": 1, "introduced": dt.date(2025, 5, 3)},
    {"country": "France", "coverage": 1, "introduced": dt.date(2025, 1, 18)},
    {"country": "Germany", "coverage": 1, "introduced": dt.date(2023, 12, 27)},
    {"country": "Greece", "coverage": 1, "introduced": dt.date(2025, 10, 23)},
    {"country": "Hungary", "coverage": 1, "introduced": dt.date(2025, 11, 19)},
    {"country": "Iceland", "coverage": 1, "introduced": dt.date(2021, 9, 9)},
    {"country": "Ireland", "coverage": 1, "introduced": dt.date(2023, 12, 14)},
    {"country": "Italy", "coverage": 1, "introduced": dt.date(2025, 3, 6)},
    {"country": "Kosovo", "coverage": 1, "introduced": dt.date(2025, 10, 25)},
    {"country": "Latvia", "coverage": 1, "introduced": dt.date(2025, 8, 28)},
    {"country": "Lithuania", "coverage": 1, "introduced": dt.date(2025, 9, 23)},
    {"country": "Luxembourg", "coverage": 1, "introduced": dt.date(2023, 12, 27)},
    {"country": "Moldova", "coverage": 1, "introduced": dt.date(2025, 11, 21)},
    {"country": "Montenegro", "coverage": 1, "introduced": dt.date(2025, 12, 18)},
    {"country": "Netherlands", "coverage": 1, "introduced": dt.date(2023, 12, 11)},
    {"country": "North Macedonia", "coverage": 1, "introduced": dt.date(2025, 12, 18)},
    {"country": "Norway", "coverage": 1, "introduced": dt.date(2021, 8, 26)},
    {"country": "Poland", "coverage": 1, "introduced": dt.date(2025, 9, 10)},
    {"country": "Portugal", "coverage": 1, "introduced": dt.date(2025, 7, 17)},
    {
        "country": "Republic of Serbia",
        "coverage": 1,
        "introduced": dt.date(2025, 10, 25),
    },
    {"country": "Romania", "coverage": 1, "introduced": dt.date(2025, 11, 21)},
    {"country": "Slovakia", "coverage": 1, "introduced": dt.date(2025, 10, 18)},
    {"country": "Slovenia", "coverage": 1, "introduced": dt.date(2025, 10, 30)},
    {"country": "Spain", "coverage": 1, "introduced": dt.date(2025, 3, 19)},
    {"country": "Sweden", "coverage": 1, "introduced": dt.date(2021, 9, 8)},
    {"country": "Switzerland", "coverage": 1, "introduced": dt.date(2023, 12, 27)},
    {"country": "Ukraine", "coverage": 1, "introduced": dt.date(2025, 10, 22)},
    {"country": "United Kingdom", "coverage": 1, "introduced": dt.date(2023, 12, 14)},
]

In [ ]:
# Create coverage dataframe

coverage_mapping = {0: "Not included in EuroEval yet", 1: "Included in EuroEval"}

# Create dataframe which contains information about EuroEval coverage
euroeval_df = pd.DataFrame.from_records(records)
euroeval_df.coverage = euroeval_df.coverage.map(coverage_mapping)

europe_df = (
    gpd.read_file("country-data/ne_110m_admin_0_countries.shp")
    .query('CONTINENT == "Europe"')
    .rename(columns=dict(ADMIN="country"))[["country", "geometry"]]
)

# Add Faroe Islands
fo_df = gpd.read_file("country-data/fo.json").rename(columns=dict(name="country"))[
    ["country", "geometry"]
]
fo_df.country = "Faroe Islands"
europe_df = pd.concat([europe_df, fo_df])

# Merge the Europe dataframe with the EuroEval dataframe
merged_df = pd.merge(
    left=europe_df, right=euroeval_df, how="outer", on="country"
).sort_values(by="country")

In [ ]:
# Create plot

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Create plot
merged_df.plot(
    column="coverage",
    cmap=cm.managua_r,
    ax=ax,
    edgecolor="black",
    linewidth=0.1,
    legend=True,
    legend_kwds=dict(loc="upper left"),
)

# Adjust to only show Europe
ax.set_xlim(-24, 41)
ax.set_ylim(35, 71)

# Remove axes
ax.axis("off")

# Add title
ax.set_title("EuroEval Coverage", fontsize=16)

# Show plot
plt.tight_layout()
plt.savefig("euroeval_coverage.png", dpi=300)
plt.show()

In [ ]:
# Create animation

FPS = 30

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Adjust to only show Europe
ax.set_xlim(-24, 41)
ax.set_ylim(35, 71)

# Remove axes
ax.axis("off")

# Get all frames, which in our case are dates
earliest_language = merged_df[~merged_df.introduced.isna()].introduced.min()
latest_language = merged_df[~merged_df.introduced.isna()].introduced.max()

frames: list[dt.date] = [earliest_language]
date = earliest_language
while date <= latest_language:
    date_with_next_change = merged_df.query("introduced > @date").introduced.min()
    if pd.isna(date_with_next_change):
        date_with_next_change = dt.date.today()
    days_to_next_change = (date_with_next_change - date).days
    days_offset = 30 if days_to_next_change > 30 else 1
    date = date + dt.timedelta(days=days_offset)
    frames.append(date)
frames += [dt.date.today()] * (3 * FPS)


@cache
def get_updated_plot(max_date: dt.date) -> Axes:
    """Cached function that gets an updated version of the plot."""
    df_plot = merged_df.copy()
    df_plot.coverage = [
        coverage_mapping[0]
        if pd.isna(row.introduced) or row.introduced > max_date
        else row.coverage
        for _, row in df_plot.iterrows()
    ]
    return df_plot.plot(
        column="coverage",
        cmap=cm.managua_r,
        ax=ax,
        edgecolor="black",
        linewidth=0.1,
        legend=True,
        legend_kwds=dict(loc="upper left"),
    )


def animation_function(frame: dt.date, *args) -> Axes:
    """Function that updates the animation."""
    max_date = max(
        row.introduced
        for _, row in merged_df.iterrows()
        if not pd.isna(row.introduced) and row.introduced <= frame
    )
    return get_updated_plot(max_date=max_date).set_title(
        f"EuroEval Coverage - {frame.isoformat()}", fontsize=16
    )


animation = anim.FuncAnimation(fig=fig, func=animation_function, frames=frames)
with tqdm(total=len(frames), desc="Saving animation") as pbar:
    animation.save(
        "euroeval_evolution.mp4",
        writer="ffmpeg",
        fps=FPS,
        dpi=300,
        progress_callback=lambda _i, _n: pbar.update(),
    )